In [49]:
import pandas as pd
import geopandas as gpd
import zipfile

In [50]:
file_names = [
    "USPPFlowMonitoring 01_2020.xls",
    "USPPFlowMonitoring 01_2021.xlsx",
    "USPPFlowMonitoring 1_2022.xlsx",
    "USPPFlowMonitoring 4_2022.xlsx",
    "USPPFlowMonitoring 5_2023.xlsx",
    "USPPFlowMonitoring 07_2020.xlsx",
    "USPPFlowMonitoring 07_2021.xlsx",
    "USPPFlowMonitoring 8_2022.xlsx",
    "USPPFlowMonitoring 10_2020.xlsx",
    "USPPFlowMonitoring 10_2021.xlsx",
    "USPPFlowMonitoring 12_2019.xls",
    "USPPFlowMonitoring 12_2020.xls",
    "USPPFlowMonitoring_7_2019.xls",
    "USPPFlowMonitoring_2018.xls",
    "USPPFlowMonitoring_Original.xls",
    "USPPFlowMonitoring_Updated2016.xls"
]

dfs = []

for name in file_names:
    sheets = pd.read_excel(name, sheet_name = None, header = 2)
    for sheet_name, df in sheets.items():
        df["Site"] = sheet_name
        df["File Name"] = name
        dfs.append(df)

In [51]:
flow = pd.concat(dfs, ignore_index = True)
flow.columns

Index(['Unnamed: 0', 'Date', 'DOY', 'Visit', 'Flow Condition', 'Flow Code',
       'Flow Trend', 'Notes', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Site', 'File Name', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')

In [52]:
flow = flow.iloc[:, [1, 5, 7, 13, 14]]
flow.head()

,Date,Flow Code,Notes,Site,File Name
0,NaN,NaN,NaN,StDavid,USPPFlowMonitoring 01_2020.xls
1,2006-01-01 00:00:00,NaN,no data=11,StDavid,USPPFlowMonitoring 01_2020.xls
2,2006-01-02 00:00:00,NaN,NaN,StDavid,USPPFlowMonitoring 01_2020.xls
3,2006-01-03 00:00:00,NaN,NaN,StDavid,USPPFlowMonitoring 01_2020.xls
4,2006-01-04 00:00:00,NaN,NaN,StDavid,USPPFlowMonitoring 01_2020.xls


In [53]:
flow.shape

(665923, 5)

In [54]:
flow.dtypes

Date         object
Flow Code    object
Notes        object
Site         object
File Name    object
dtype: object

In [55]:
flow["Date"] = flow["Date"].astype(str).str.replace("02/29/18", "02/28/18") # 02/29/18 not a leap year
flow["Date"] = flow["Date"].astype(str).str.replace("\\", "/")
flow["Date"] = flow["Date"].astype(str).str.replace("1/9/205", "1/9/2025") # Date entered as 1/9/205, year manually verified
flow["Date"] = pd.to_datetime(flow["Date"], format='mixed', dayfirst=False)
flow = flow.drop_duplicates(subset = ["Date", "Site"], keep = "first")
flow = flow.sort_values(["Date", "Site"]).reset_index(drop = True)


In [56]:
flow.shape

(58178, 5)

In [57]:
kmz = zipfile.ZipFile("Streamflow Cameras.kmz", "r").extractall(".")

In [58]:
location = gpd.read_file("doc.kml", driver = "KML") # driver specifies file format
location["Longitude"] = location.geometry.x
location["Latitude"] = location.geometry.y
location["Name"] = location["Name"].str.strip().str.removesuffix(" Camera")


In [59]:
flow["Site"].unique()

array(['Boquillas', 'Char_Mesq', 'Contention', 'Fairbank', 'Hereford',
       'Hunter', 'Moson', 'StDavid', 'Escalante', 'FairbankNorth',
       'Cottonwood', 'LewisSprings'], dtype=object)

In [60]:
flow["Site"] = flow["Site"].replace("StDavid", "St.David")
flow["Site"] = flow["Site"].replace("Char_Mesq", "CharlestonMesquite")

In [61]:
flow = flow.merge(
    location[["Name", "Latitude", "Longitude"]],
    left_on = "Site", right_on = "Name",
    how = "left"
)

flow = flow.drop(columns = ["Name"])
flow = flow[["Site", "Date", "Flow Code", "Latitude", "Longitude", "Notes", "File Name"]]
flow = flow.iloc[:-12].reset_index(drop = True)

In [62]:
flow.head()

,Site,Date,Flow Code,Latitude,Longitude,Notes,File Name
0,Boquillas,2006-01-01,NaN,31.689704,-110.185152,NaN,USPPFlowMonitoring 01_2020.xls
1,CharlestonMesquite,2006-01-01,NaN,31.665863,-110.178841,NaN,USPPFlowMonitoring 01_2020.xls
2,Contention,2006-01-01,NaN,31.767399,-110.205148,NaN,USPPFlowMonitoring 01_2020.xls
3,Fairbank,2006-01-01,NaN,31.722309,-110.193796,no data=11,USPPFlowMonitoring 01_2020.xls
4,Hereford,2006-01-01,NaN,31.461756,-110.109676,NaN,USPPFlowMonitoring 01_2020.xls


In [63]:
flow.to_csv("USPPFlowMonitoring2006_2025.csv", index = False)